Approach (worked - 90k records in 15 min)

In [1]:
import pandas as pd
import psycopg2
import os
import uuid
import logging
from dotenv import load_dotenv
from psycopg2 import pool
from concurrent.futures import ThreadPoolExecutor

# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create console handler and set level to INFO
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# Create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Add formatter to console handler
ch.setFormatter(formatter)

# Add console handler to logger
logger.addHandler(ch)

# Load environment variables from .env file
load_dotenv()

# Connection parameters
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
max_connections = 5  # Adjust the maximum number of connections as needed

# Function to insert data into the PostgreSQL table
def insert_data(chunk_df):
    try:
        # Get a connection from the connection pool
        conn = db_pool.getconn()
        cursor = conn.cursor()

        # Iterate over chunk DataFrame and insert data into the table
        for index, row in chunk_df.iterrows():
            species_id = str(row['species_id'])
            iucn_category_id = str(row['iucn_category_id'])
            iucn_taxon_id = str(row['iucn_taxon_id'])
            cursor.execute("""
                INSERT INTO species (
                    species_id,
                    iucn_category_id,
                    iucn_taxon_id,
                    common_name,
                    binominal_name,
                    name_language,
                    species_group,
                    total_habitat_range_area,
                    small_habitat_range_area,
                    created,
                    updated
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, current_timestamp, current_timestamp
                )""", (
                    species_id,
                    iucn_category_id,
                    iucn_taxon_id,
                    row['common_name'],
                    row['binomial'],
                    row['name_language'],
                    row['group'], 
                    row['total_area'],
                    row['small_range']
                )
            )
        # Commit the transaction
        conn.commit()
        # Return connection to the pool
        db_pool.putconn(conn)
        logger.info(f"Inserted {len(chunk_df)} records into the database.")
    except Exception as e:
        logger.error(f"Error inserting data: {e}")
        if conn:
            conn.rollback()
            db_pool.putconn(conn)

# Create a connection pool
db_pool = psycopg2.pool.SimpleConnectionPool(minconn=1, maxconn=max_connections,
                                             dbname=dbname, user=user,
                                             password=password, host=host,
                                             port=port)

# Read the CSV file into a DataFrame
csv_file = '../data/countries_species.csv'
df = pd.read_csv(csv_file)

# Drop irrelevant columns
df.drop(columns=['iso_a3', 'wb_datanam', 'wb_iso', 'datanam_area', 'datanam_pct_area'], inplace=True)

# Fetch data from the iucn_categories table
query = "SELECT iucn_category_id, abbreviation FROM iucn_categories"
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port) as conn:
    with conn.cursor() as cursor:
        cursor.execute(query)
        iucn_categories = cursor.fetchall()

# Create a dictionary for mapping iucn_category abbreviations to ids
iucn_category_mapping = {abbreviation: iucn_category_id for iucn_category_id, abbreviation in iucn_categories}

# Map iucn_category_id to the DataFrame using 'iucn_category' column
df['iucn_category_id'] = df['iucn_category'].map(iucn_category_mapping)

# Drop the 'iucn_category' column as it's no longer needed
df.drop(columns=['iucn_category'], inplace=True)

# Rename 'iucn_id_no' column to 'iucn_taxon_id' to match the table schema
df.rename(columns={'iucn_id_no': 'iucn_taxon_id'}, inplace=True)

df['iucn_taxon_id'] = df['iucn_taxon_id'].astype(str).str.replace('\.0', '', regex=True)

# Replace missing values in 'iucn_category_id' with the UUID corresponding to 'DD' abbreviation
dd_uuid_query = "SELECT iucn_category_id FROM iucn_categories WHERE abbreviation = 'DD'"
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port) as conn:
    with conn.cursor() as cursor:
        cursor.execute(dd_uuid_query)
        dd_uuid = cursor.fetchone()[0]

df['iucn_category_id'].fillna(dd_uuid, inplace=True)

# Drop duplicates based on relevant columns
relevant_columns = ['group', 'binomial', 'common_name', 'name_language', 'total_area', 'small_range']
df.drop_duplicates(subset=relevant_columns, keep='first', inplace=True)

# Generate UUIDs for the species_id column
df['species_id'] = [uuid.uuid4() for _ in range(len(df))]

# # Save DataFrame to CSV file
# csv_output_file = '../data/countries_species_processed.csv'
# df.to_csv(csv_output_file, index=False)

# logger.info(f"Processed data saved to {csv_output_file}")

# Insert data into the PostgreSQL table in parallel using ThreadPoolExecutor
chunk_size = 50
total_records = len(df)
logger.info(f"Total records to insert: {total_records}")
with ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, total_records, chunk_size):
        chunk_df = df[i:i+chunk_size]
        futures.append(executor.submit(insert_data, chunk_df))
    for future in futures:
        future.result()

logger.info("Data insertion completed successfully.")


C:\Users\conno\AppData\Local\Temp\ipykernel_10296\2837328796.py:96: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
2024-03-02 17:00:23,750 - INFO - Total records to insert: 91220
2024-03-02 17:00:31,058 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:32,709 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:33,058 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:33,366 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:33,984 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:34,065 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:34,742 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:34,742 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:36,198 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:37,760 - INFO - Inserted 50 records into the database.
2024-03-02 17:00:

Save output to file for next step

In [1]:
import pandas as pd
import psycopg2
import os
import uuid
import logging
from dotenv import load_dotenv
from psycopg2 import pool
from concurrent.futures import ThreadPoolExecutor

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables from .env file
load_dotenv()

# Connection parameters
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
max_connections = 5  # Adjust the maximum number of connections as needed

# Function to insert data into the PostgreSQL table
def insert_data(chunk_df):
    try:
        # Get a connection from the connection pool
        conn = db_pool.getconn()
        cursor = conn.cursor()

        # Iterate over chunk DataFrame and insert data into the table
        for index, row in chunk_df.iterrows():
            species_id = str(row['species_id'])
            iucn_category_id = str(row['iucn_category_id'])
            iucn_taxon_id = str(row['iucn_taxon_id'])
            cursor.execute("""
                INSERT INTO species (
                    species_id,
                    iucn_category_id,
                    iucn_taxon_id,
                    common_name,
                    binominal_name,
                    name_language,
                    species_group,
                    total_habitat_range_area,
                    small_habitat_range_area,
                    created,
                    updated
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, current_timestamp, current_timestamp
                )""", (
                    species_id,
                    iucn_category_id,
                    iucn_taxon_id,
                    row['common_name'],
                    row['binomial'],
                    row['name_language'],
                    row['group'], 
                    row['total_area'],
                    row['small_range']
                )
            )
        # Commit the transaction
        conn.commit()
        # Return connection to the pool
        db_pool.putconn(conn)
        logger.info(f"Inserted {len(chunk_df)} records into the database.")
    except Exception as e:
        logger.error(f"Error inserting data: {e}")
        if conn:
            conn.rollback()
            db_pool.putconn(conn)

# Create a connection pool
db_pool = psycopg2.pool.SimpleConnectionPool(minconn=1, maxconn=max_connections,
                                             dbname=dbname, user=user,
                                             password=password, host=host,
                                             port=port)

# Read the CSV file into a DataFrame
csv_file = '../data/countries_species.csv'
df = pd.read_csv(csv_file)

# Drop irrelevant columns
df.drop(columns=['iso_a3', 'wb_datanam', 'wb_iso', 'datanam_area', 'datanam_pct_area'], inplace=True)

# Fetch data from the iucn_categories table
query = "SELECT iucn_category_id, abbreviation FROM iucn_categories"
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port) as conn:
    with conn.cursor() as cursor:
        cursor.execute(query)
        iucn_categories = cursor.fetchall()

# Create a dictionary for mapping iucn_category abbreviations to ids
iucn_category_mapping = {abbreviation: iucn_category_id for iucn_category_id, abbreviation in iucn_categories}

# Map iucn_category_id to the DataFrame using 'iucn_category' column
df['iucn_category_id'] = df['iucn_category'].map(iucn_category_mapping)

# Drop the 'iucn_category' column as it's no longer needed
df.drop(columns=['iucn_category'], inplace=True)

# Rename 'iucn_id_no' column to 'iucn_taxon_id' to match the table schema
df.rename(columns={'iucn_id_no': 'iucn_taxon_id'}, inplace=True)

df['iucn_taxon_id'] = df['iucn_taxon_id'].astype(str).str.replace('\.0', '', regex=True)

# Replace missing values in 'iucn_category_id' with the UUID corresponding to 'DD' abbreviation
dd_uuid_query = "SELECT iucn_category_id FROM iucn_categories WHERE abbreviation = 'DD'"
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port) as conn:
    with conn.cursor() as cursor:
        cursor.execute(dd_uuid_query)
        dd_uuid = cursor.fetchone()[0]

df['iucn_category_id'].fillna(dd_uuid, inplace=True)

# Drop duplicates based on relevant columns
relevant_columns = ['group', 'binomial', 'common_name', 'name_language', 'total_area', 'small_range']
df.drop_duplicates(subset=relevant_columns, keep='first', inplace=True)

# Generate UUIDs for the species_id column
df['species_id'] = [uuid.uuid4() for _ in range(len(df))]

# Save DataFrame to CSV file
csv_output_file = '../data/countries_species_processed.csv'
df.to_csv(csv_output_file, index=False)

logger.info(f"Processed data saved to {csv_output_file}")

# Insert data into the PostgreSQL table in parallel using ThreadPoolExecutor
chunk_size = 50
total_records = len(df)
logger.info(f"Total records to insert: {total_records}")
with ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, total_records, chunk_size):
        chunk_df = df[i:i+chunk_size]
        futures.append(executor.submit(insert_data, chunk_df))
    for future in futures:
        future.result()

logger.info("Data insertion completed successfully.")
